In [1]:
import os
import pickle
import cv2
import mediapipe as mp
import numpy as np
import time
from PIL import ImageFont, ImageDraw, Image
from unicode import join_jamos, split_syllables, join_jamos_char # 자음+모음+자음, 쌍자음, 겹받침 도와주는 라이브러리
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report




print("프로그램 실행 중...")
#-------------------------------------------------------------------------------------------------------------------------------
# 저장된 모델 파일을 열고 불러오기
model_dict = pickle.load(open('C:/Users/kovin/Desktop/sign-final/model.p', 'rb'))
model = model_dict['model']

# 웹캠을 사용하여 비디오 캡처
cap = cv2.VideoCapture(0)

# 창 이름 설정
window_name = "Sign Language Recognition"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

# 원하는 크기로 창 크기 설정 (예: 1280x800 + 오른쪽 270px 추가)
cv2.resizeWindow(window_name, 1550, 800)


#-------------------------------------------------------------------------------------------------------------------------------

mp_hands = mp.solutions.hands  # MediaPipe 손 솔루션
mp_drawing = mp.solutions.drawing_utils  # MediaPipe 그림 도구
mp_drawing_styles = mp.solutions.drawing_styles  # MediaPipe 그림 스타일
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3, max_num_hands=2)  # 손 추적 초기화

#-------------------------------------------------------------------------------------------------------------------------------

# 한글 폰트 설정
font_path = 'C:/Windows/Fonts/Hancom Gothic Bold.ttf'
font_size = 50
font_size_log = 10
font = ImageFont.truetype(font_path, font_size)  # PIL에서 사용할 폰트 로드
font_log = ImageFont.truetype(font_path, font_size_log) # 로그 창에서 사용할 폰트

#-------------------------------------------------------------------------------------------------------------------------------
# 로그를 저장할 리스트
log_messages = []

def add_log_message(message):
    """로그 메시지를 추가하고 최대 24줄로 유지"""
    log_messages.append(message)
    if len(log_messages) > 24:
        log_messages.pop(0)  # 오래된 메시지 삭제

#-------------------------------------------------------------------------------------------------------------------------------


# 각 레이블에 해당하는 한글 문자 딕셔너리
labels_dict = {0: 'ㄱ', 1: 'ㄴ', 2: 'ㄷ', 3: 'ㄹ', 4: 'ㅁ', 5: 'ㅂ', 6: 'ㅅ', 7: 'ㅇ', 8: 'ㅈ', 9: 'ㅊ', 10: 'ㅋ', 11: 'ㅌ', 12: 'ㅍ', 13: 'ㅎ',
               14: 'ㅏ', 15: 'ㅑ', 16: 'ㅓ', 17: 'ㅕ', 18: 'ㅗ', 19: 'ㅛ', 20: 'ㅜ', 21: 'ㅠ', 22: 'ㅡ', 23: 'ㅣ', 24: 'ㅐ', 25: 'ㅒ', 26: 'ㅔ', 27: 'ㅖ',
               28: 'ㅢ', 29: 'ㅚ', 30: 'ㅟ', 31: '', 32: '', 33: '', 34: '', 35: '', 36: '', 37: '', 38: ''
               , 39: '', 40: '', 41: '', 42: '', 43: '', 44: '',
               45: '', 46: '', 47: '', 48: '', 49: '', 50: '', 51: '', 52: ''}

# 31: '긍정1', 32: '긍정2', 33: '부정1', 34: '부정2', 35: '감사1', 36: '감사2', 37: '안녕1', 38: '안녕2',
# 39: '기쁘다1', 40: '기쁘다2', 41: '화나다1', 42: '화나다2', 43: '반갑다1', 44: '반갑다2',
# 45: '슬프다1', 46: '슬프다2', 47: '아니다1', 48: '아니다2', 49: '묻다1', 50: '묻다2', 51: '모르다1', 52: '모르다2'


# 연속된 동작 딕셔너리
sequential_actions = {'긍정': [31, 32], '부정': [33, 34], '감사': [35, 36], '안녕': [37, 38],
                      '기쁘다': [39, 40], '화나다': [41, 42], '반갑다': [43, 44], '슬프다': [45, 46],
                      '아니다': [47, 48], '묻다': [31, 50], '모르다': [51, 52]}


#-------------------------------------------------------------------------------------------------------------------------------


# 특정 조인트에 가중치를 적용하는 함수
def apply_joint_weights(data, weights):
    weighted_data = data.copy()
    for idx, weight in weights.items():
        if idx * 3 + 2 < len(weighted_data):  # 인덱스 범위 체크
            weighted_data[idx * 3] *= weight
            weighted_data[idx * 3 + 1] *= weight
            weighted_data[idx * 3 + 2] *= weight
    return weighted_data
    
finger_tip_joints = [12, 20, 11, 19, 10]  # 중지, 새끼의 중간과 끝 관절에
weights = {joint: 1.5 for joint in finger_tip_joints}  # 가중치 1.5 적용

#-------------------------------------------------------------------------------------------------------------------------------


# 이전에 인식된 동작을 저장할 변수
previous_action = None
previous_action_2 = None

# 연속 동작 인식 중간에 다른 동작이 인식된 경우를 처리할 변수
intermediate_action = None
intermediate_action_time = None

# 이전 동작 인식 시간 저장 변수
previous_action_time = None

# 현재 스택에 저장된 수어를 보여주기 위한 변수
current_gesture_stack = []

# 이전에 표시된 텍스트
previous_text = ""

is_combined = 0  # 현재 글자가 합쳐진 글자인지 확인하는 변수
is_double_consonant = 0 # 이중자음 결합 여부를 추적하는 변수 추가

#-------------------------------------------------------------------------------------------------------------------------------

# 이미지에 텍스트를 추가하는 함수
def put_text_with_font(image, text, position, font, font_size, color, thickness):
    img_pil = Image.fromarray(image)  # OpenCV 이미지를 PIL 이미지로 변환
    draw = ImageDraw.Draw(img_pil)  # PIL 이미지에 그리기 객체 생성
    draw.text(position, text, font=font, fill=color)  # 지정된 위치에 텍스트 그리기
    return np.array(img_pil)  # PIL 이미지를 다시 OpenCV 이미지로 변환
    

#-------------------------------------------------------------------------------------------------------------------------------

def process_gesture_stack(current_gesture_stack, is_combined, is_double_consonant):

    double_consonant_map = {
        # 단순 이중자음
        ('ㄱ', 'ㄱ'): 'ㄲ',
        ('ㄷ', 'ㄷ'): 'ㄸ',
        ('ㅂ', 'ㅂ'): 'ㅃ',
        ('ㅅ', 'ㅅ'): 'ㅆ',
        ('ㅈ', 'ㅈ'): 'ㅉ',
        # 복합 이중자음
        ('ㄱ', 'ㅅ'): 'ㄳ',
        ('ㄴ', 'ㅈ'): 'ㄵ',
        ('ㄴ', 'ㅎ'): 'ㄶ',
        ('ㄹ', 'ㄱ'): 'ㄺ',
        ('ㄹ', 'ㅁ'): 'ㄻ',
        ('ㄹ', 'ㅂ'): 'ㄼ',
        ('ㄹ', 'ㅅ'): 'ㄽ',
        ('ㄹ', 'ㅌ'): 'ㄾ',
        ('ㄹ', 'ㅍ'): 'ㄿ',
        ('ㄹ', 'ㅎ'): 'ㅀ',
        ('ㅂ', 'ㅅ'): 'ㅄ',
    }

    if prediction is not None:
        predicted_character = labels_dict.get(int(prediction[0]), None)

        # 예측된 값이 0~30 범위에 있는지 확인
        if 0 <= int(prediction[0]) <= 30:
            current_gesture_stack.append(predicted_character)  # 현재 제스처 스택에 예측된 문자를 추가

            # 이중자음 처리 (is_combined과 독립적)
            if len(current_gesture_stack) >= 2 and is_double_consonant == 0:
                last_two = (current_gesture_stack[-2], current_gesture_stack[-1])
                if last_two in double_consonant_map:
                    combined_char = double_consonant_map[last_two]
                    current_gesture_stack.pop()
                    current_gesture_stack.pop()
                    current_gesture_stack.append(combined_char)
                    is_double_consonant = 1

            # 종성에서 이중자음 처리
            if len(current_gesture_stack) >= 3 and is_double_consonant == 0:
                try:
                    init, med, final = split_syllables(current_gesture_stack[-2])
                except ValueError:
                    pass  # 한글 음절이 아닌 경우
                else:
                    # 종성 결합 가능한지 확인
                    if (final, current_gesture_stack[-1]) in double_consonant_map:
                        new_final = double_consonant_map[(final, current_gesture_stack[-1])]
                        combined_char = join_jamos(init + med + new_final)
                        current_gesture_stack.pop()  # 마지막 문자 제거
                        current_gesture_stack.pop()  # 기존 음절 제거
                        current_gesture_stack.append(combined_char)
                        is_double_consonant = 1

            # 초성-중성 결합
            if is_combined == 0 and len(current_gesture_stack) >= 2:
                if current_gesture_stack[-2] in ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㄲ', 'ㄸ', 'ㅃ', 'ㅆ', 'ㅉ'] and current_gesture_stack[-1] in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅐ', 'ㅒ', 'ㅔ', 'ㅖ', 'ㅢ', 'ㅚ']:
                    combined_text = join_jamos(''.join(current_gesture_stack[-2:]))
                    current_gesture_stack.pop()
                    current_gesture_stack.pop()
                    current_gesture_stack.append(combined_text)
                    is_combined = 1

            elif is_combined == 1 and current_gesture_stack[-1] in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅐ', 'ㅒ', 'ㅔ', 'ㅖ', 'ㅢ', 'ㅚ']:
                is_combined = 0

            elif is_combined == 1 and current_gesture_stack[-1] in ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㄲ', 'ㄸ', 'ㅃ', 'ㅆ', 'ㅉ', 'ㄳ', 'ㄵ', 'ㄶ', 'ㄺ', 'ㄻ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅄ']:
                try:
                    init, med = split_syllables(current_gesture_stack[-2])
                except ValueError:
                    pass
                else:
                    removed_value = current_gesture_stack.pop()
                    current_gesture_stack.pop()
                    current_gesture_stack.append(init)
                    current_gesture_stack.append(med)
                    current_gesture_stack.append(removed_value)

                    if len(current_gesture_stack) >= 3:
                        if current_gesture_stack[-3] in ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㄲ', 'ㄸ', 'ㅃ', 'ㅆ', 'ㅉ']:
                            if current_gesture_stack[-2] in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅐ', 'ㅒ', 'ㅔ', 'ㅖ', 'ㅢ', 'ㅚ']:
                                if current_gesture_stack[-1] in ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㄲ', 'ㅆ', 'ㄳ', 'ㄵ', 'ㄶ', 'ㄺ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅄ']:
                                    combined_text = join_jamos(''.join(current_gesture_stack[-3:]))
                                    current_gesture_stack.pop()
                                    current_gesture_stack.pop()
                                    current_gesture_stack.pop()
                                    current_gesture_stack.append(combined_text)
                                    is_combined = 0

        return current_gesture_stack, is_combined, is_double_consonant


#-------------------------------------------------------------------------------------------------------------------------------

# 동작별 타이머 및 상태 변수 초기화
action_states = {
    '띄어쓰기': {'start_time': None, 'count': 0},
    '한 글자 삭제': {'start_time': None, 'count': 0},
    '스택 초기화': {'start_time': None, 'count': 0}
}


def perform_action(action_name):
    """
    각 동작에 따라 실제 수행 작업을 처리하는 함수
    :param action_name: 수행할 동작의 이름
    """
    global is_combined
    global current_gesture_stack
    
    if action_name == "띄어쓰기":
        # '띄어쓰기' 동작 수행
        is_combined = 0  # is_combined 값을 초기화
        is_double_consonant = 0
        print("띄어쓰기 동작: is_combined 초기화 완료")
        add_log_message("is_combined 초기화 동작 : 글자 간격을 나누었습니다.")
        
    elif action_name == "한 글자 삭제":
        # '한 글자 삭제' 동작 수행
        if current_gesture_stack:  # 스택이 비어있지 않을 때만 실행
            current_gesture_stack.pop()  # 스택에서 마지막 문자를 제거
            print("한 글자 삭제 동작: 스택에서 마지막 문자가 삭제됨")
            add_log_message("한 글자 삭제 동작: 스택에서 마지막 문자가 삭제되었습니다.")
        else:
            print("한 글자 삭제 동작: 스택이 비어 있음")
            add_log_message("한 글자 삭제 동작: 현재 스택이 비어 있습니다.")
            
    elif action_name == "스택 초기화":
        # '스택 초기화' 동작 수행
        current_gesture_stack = []  # 스택 비우기
        print("스택 초기화 동작: 스택이 초기화됨")
        add_log_message("스택 초기화 동작 : 글자를 모두 지웠습니다.")
    else:
        print(f"알 수 없는 동작: {action_name}")


def process_action_independently(prediction, action_code, duration, action_name):
    """
    각 동작에 대해 독립적인 처리 시간을 관리하며 동작 수행을 처리하는 함수
    :param prediction: 모델 예측 결과
    :param action_code: 동작 코드
    :param duration: 동작 감지 지속 시간 (초)
    :param action_name: 동작 이름 (수행 작업용)
    """
    global action_states
    current_time = time.time()
    action_state = action_states[action_name]
    # 동작 감지 여부 확인
    is_detected = int(prediction[0]) == action_code
    # 동작이 감지되었을 경우
    if is_detected:
        # 타이머 초기화 여부 확인
        if action_state['start_time'] is None:
            action_state['start_time'] = current_time
        action_state['count'] += 1  # 카운터 증가
        # 설정된 지속 시간이 경과했을 경우 동작 수행
        if current_time - action_state['start_time'] >= duration:
            total_actions = action_state['count']
            if total_actions > 0:
                perform_action(action_name)  # 동작 수행
            # 동작 상태 초기화
            action_state['count'] = 0
            action_state['start_time'] = None
    # 동작이 감지되지 않았을 경우
    else:
        # 상태 초기화
        action_state['count'] = 0
        action_state['start_time'] = None


#-------------------------------------------------------------------------------------------------------------------------------
MAX_STACK_LENGTH = 12  # 텍스트 스택 최대 길이

def manage_stack_length(stack):
    """
    스택의 길이를 제한하는 함수. 초과 시 오래된 항목을 삭제.
    :param stack: 현재 텍스트 스택
    :return: 수정된 스택
    """
    if len(stack) > MAX_STACK_LENGTH:
        stack.pop(0)  # 스택의 첫 번째 요소 제거
    return stack

#-------------------------------------------------------------------------------------------------------------------------------


# 인식 결과를 카운팅하는 딕셔너리
result_counts = {}
last_check_time = time.time()
is_hand_detected = False
start_time = None  # n초 타이머 시작 시간
total_frames = 0


#-------------------------------------------------------------------------------------------------------------------------------

while cap.isOpened():
    data_aux = []  # 데이터 보조 리스트 초기화
    x_ = []  # x 좌표 리스트 초기화
    y_ = []  # y 좌표 리스트 초기화
    z_ = []  # z 좌표 리스트 초기화
    ret, frame = cap.read()  # 비디오 프레임 읽기
    if not ret:
        break
    H, W, _ = frame.shape  # 프레임의 높이, 너비, 채널 가져오기
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # 프레임을 RGB로 변환
    results = hands.process(frame_rgb)  # 손 추적 결과
    if results.multi_hand_landmarks:  # 손이 감지된 경우
        # 손이 인식되면 타이머 시작
        if start_time is None:
            start_time = time.time()
        for hand_landmarks in results.multi_hand_landmarks:  # 각 손에 대해
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                      mp_drawing_styles.get_default_hand_landmarks_style(),
                                      mp_drawing_styles.get_default_hand_connections_style())  # 손 랜드마크 및 연결 그리기
            for i in range(len(hand_landmarks.landmark) - 1):  # 각 랜드마크에 대해
                x1, y1, z1 = hand_landmarks.landmark[i].x, hand_landmarks.landmark[i].y, hand_landmarks.landmark[i].z
                x2, y2, z2 = hand_landmarks.landmark[i + 1].x, hand_landmarks.landmark[i + 1].y, hand_landmarks.landmark[i + 1].z
                vx = x2 - x1  # 벡터 x 성분
                vy = y2 - y1  # 벡터 y 성분
                vz = z2 - z1  # 벡터 z 성분
                norm = np.sqrt(vx ** 2 + vy ** 2 + vz ** 2)  # 벡터의 크기 계산
                vx /= norm  # 벡터 정규화
                vy /= norm
                vz /= norm
                data_aux.extend([vx, vy, vz])  # 정규화된 벡터 성분 추가
                
            data_aux = apply_joint_weights(np.array(data_aux), weights).tolist()  # 가중치 적용
                
            x1 = int(min([lm.x for lm in hand_landmarks.landmark]) * W) - 10
            y1 = int(min([lm.y for lm in hand_landmarks.landmark]) * H) - 10
            x2 = int(max([lm.x for lm in hand_landmarks.landmark]) * W) + 10
            y2 = int(max([lm.y for lm in hand_landmarks.landmark]) * H) + 10
            
            prediction = model.predict([np.asarray(data_aux)])  # 모델을 사용하여 예측
            min_time_threshold = 0.5  # 0.5초 이하의 동작은 무시
            if prediction is not None:
                predicted_character = labels_dict[int(prediction[0])]  # 예측된 문자를 라벨 딕셔너리에서 가져옴
                process_action_independently(prediction, 46, 1, "띄어쓰기")
                process_action_independently(prediction, 50, 1, "한 글자 삭제")
                process_action_independently(prediction, 38, 2, "스택 초기화") #-----------------------------------!! 기능 사용을 위한 인식 시간 조정 가능
                # 인식 결과 카운팅
                result_counts[predicted_character] = result_counts.get(predicted_character, 0) + 1
        
                # n초 마다 가장 빈도수가 높은 결과를 스택에 추가
                current_time = time.time()
                total_frames += 1  # 총 프레임 수 증가
    
                if current_time - start_time >= 1: #스택에 추가 되는 시간 (n초)
                    if result_counts:
                        total_recognized = sum(result_counts.values())
                        most_frequent_result, max_count = max(result_counts.items(), key=lambda x: x[1])
    
                        # 최소 프레임 수와 빈도 비율 조건을 만족하면 스택에 추가
                        min_frames = 10
                        if total_frames >= min_frames and max_count / total_frames >= 0.9:
                            current_gesture_stack, is_combined, is_double_consonant = process_gesture_stack(current_gesture_stack, is_combined, is_double_consonant)
                            current_gesture_stack = manage_stack_length(current_gesture_stack) # 스크롤 기능을 위함
                            
                    # n초 타이머와 관련 변수 초기화
                    start_time = time.time()  # 타이머 재시작
                    total_frames = 0  # 총 프레임 수 초기화
                    result_counts = {}  # 인식된 결과 초기화
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 4)  # 손 주위에 사각형을 그림
                frame = put_text_with_font(frame, predicted_character, (x1, y1 - 70), font, font_size, (255, 0, 255), 5)  # 예측된 문자를 프레임에 그림
                
                current_time = time.time()  # 현재 시간을 저장
                # 연속 동작의 인식 및 처리
                if previous_action is not None:
                    if current_time - previous_action_time <= 3:
                        sequence_found = False
                        for action, sequence in sequential_actions.items():
                            # 양방향 순서 인식 가능하도록 수정
                            if (previous_action == sequence[0] and int(prediction[0]) == sequence[1]) or \
                               (previous_action == sequence[1] and int(prediction[0]) == sequence[0]):
                                current_text = action
                                if previous_text != current_text:
                                    previous_text = current_text
                                sequence_found = True
                                break
                
                        if sequence_found:
                            previous_action_2 = previous_action
                            previous_action = None
                            previous_action_time = None
                        else:
                            pass  # 잘못된 동작이지만 무시
                    else:
                        previous_action = None
                        previous_action_time = None
                
                # 연속 동작의 시작 동작인지 확인
                sequence_started = False
                if previous_action_time is None or (current_time - previous_action_time >= min_time_threshold):
                    for action, sequence in sequential_actions.items():
                        if int(prediction[0]) == sequence[0]:
                            previous_action = sequence[0]
                            previous_action_time = current_time
                            sequence_started = True
                            break
                        elif int(prediction[0]) == sequence[1]:  # 양방향 인식 추가
                            previous_action = sequence[1]
                            previous_action_time = current_time
                            sequence_started = True
                            break
                            
            # 스택에 수어 저장 및 출력
            key = cv2.waitKey(1)
            if key == 13:  # Enter 키가 눌렸을 때
                current_gesture_stack, is_combined, is_double_consonant = process_gesture_stack(current_gesture_stack, is_combined, is_double_consonant)
                current_gesture_stack = manage_stack_length(current_gesture_stack) # 스크롤 기능을 위함
                add_log_message("글자 입력 동작 : 글자가 입력 되었습니다.")
                
            elif key == 32:  # Space 키가 눌렸을 때
                # '띄어쓰기' 동작 수행
                is_combined = 0  # is_combined 값을 초기화
                is_double_consonant = 0
                print("띄어쓰기 동작: is_combined 초기화 완료")
                add_log_message("is_combined 초기화 동작 : 글자 간격을 나누었습니다.")
                # '전우진' 같은경우 '전우(스페이스바)진' 가능,, 안하면 '전웆ㅣㄴ'됨..
        
            
                    
            elif key == 8:  # Backspace 키가 눌렸을 때
                # '한 글자 삭제' 동작 수행
                if current_gesture_stack:  # 스택이 비어있지 않을 때만 실행
                    current_gesture_stack.pop()  # 스택에서 마지막 문자를 제거
                    print("한 글자 삭제 동작: 스택에서 마지막 문자가 삭제됨")
                    add_log_message("한 글자 삭제 동작: 스택에서 마지막 문자가 삭제되었습니다.")
                else:
                    print("한 글자 삭제 동작: 스택이 비어 있음")
                    add_log_message("한 글자 삭제 동작: 현재 스택이 비어 있습니다.")
                    
            data_aux = []  # 특징점을 저장할 리스트 초기화
            x_ = []  # x 좌표를 저장할 리스트 초기화
            y_ = []  # y 좌표를 저장할 리스트 초기화
            z_ = []  # z 좌표를 저장할 리스트 초기화
#-------------------------------------------------------------------------------------------------------------------------------

    
    # 현재 스택에 저장된 수어를 보여주기 위한 변수
    stack_text = ''.join(current_gesture_stack)

    # 기존 오리지널 출력 : 배경 X
    # # 현재 스택에 저장된 수어 출력
    # frame = put_text_with_font(frame, stack_text, (50, H - 70), font, font_size, (255, 255, 0), 5)
    
    # # 연속된 동작 텍스트 출력
    # if previous_text:
    #     frame = put_text_with_font(frame, previous_text, (50, 70), font, font_size, (0, 255, 0), 5)

    # 현재 스택에 저장된 수어 출력
    # 고정된 배경 크기와 위치 설정
    background_x, background_y = 0, H - 60  # 배경 시작 좌표
    background_width, background_height = 639, 58  # 배경 크기 (너비, 높이)

    # 배경 사각형과 테두리
    cv2.rectangle(frame, 
                  (background_x, background_y), 
                  (background_x + background_width, background_y + background_height), 
                  (255, 248, 240), -1)  # 배경 (RGB) : 240, 248, 255
    cv2.rectangle(frame, 
                  (background_x, background_y), 
                  (background_x + background_width, background_y + background_height), 
                  (235, 71, 112), 2)  # 테두리 (RGB) : 112, 71, 235

    # 텍스트 출력 (배경 중앙에 정렬)
    frame = put_text_with_font(frame, stack_text, (25, H - 65), font, font_size, (0, 0, 0), 5)


    # 연속된 동작 텍스트 출력
    # 고정된 배경 크기와 위치 설정
    background_x, background_y = 0, 0  # 배경 시작 좌표
    background_width, background_height = 160, 60  # 배경 크기 (너비, 높이)

    # 배경 사각형 (흰색)과 테두리 (검은색)
    cv2.rectangle(frame, 
                  (background_x, background_y), 
                  (background_x + background_width, background_y + background_height), 
                  (255, 248, 240), -1)  # 배경 (RGB) : 240, 248, 255
    cv2.rectangle(frame, 
                  (background_x, background_y), 
                  (background_x + background_width, background_y + background_height), 
                  (102, 102, 102), 1)  # 테두리 (RGB) : 60, 60, 144

    # 연속된 동작 텍스트 출력
    if previous_text:
        frame = put_text_with_font(frame, previous_text, (10, -5), font, font_size, (0, 255, 0), 5)

#-------------------------------------------------------------------------------------------------------------------------------

    # 로그 창 크기 설정 (270px 너비)
    log_width = 270
    frame_height, frame_width, _ = frame.shape
    combined_frame = np.zeros((frame_height, frame_width + log_width, 3), dtype=np.uint8)
    combined_frame[:, :frame_width] = frame

    # 로그 메시지 그리기
    log_img = Image.new('RGB', (log_width, frame_height), color='white')
    draw = ImageDraw.Draw(log_img)
    for i, message in enumerate(log_messages[-24:]):  # 최근 20개의 메시지만 표시
        draw.text((10, 20 * i), message, font=font_log, fill='black')
    log_array = np.array(log_img)

    # 프레임과 로그 결합
    combined_frame[:, frame_width:] = log_array
    
        
    cv2.imshow(window_name, combined_frame)  # 프레임 표시
    
    if cv2.waitKey(1) & 0xFF == 27:  # Esc 키가 눌렸을 때 루프 종료
        print("프로그램이 종료됩니다.")
        break

#-------------------------------------------------------------------------------------------------------------------------------

cap.release()  # 비디오 캡처 해제
cv2.destroyAllWindows()  # 모든 OpenCV 창 닫기

프로그램 실행 중...
띄어쓰기 동작: is_combined 초기화 완료
스택 초기화 동작: 스택이 초기화됨
띄어쓰기 동작: is_combined 초기화 완료
띄어쓰기 동작: is_combined 초기화 완료
띄어쓰기 동작: is_combined 초기화 완료
띄어쓰기 동작: is_combined 초기화 완료
프로그램이 종료됩니다.
